In [51]:
import cv2
import numpy as np
import math
from scipy import ndimage

In [52]:
img = cv2.imread('marriage.jpg')
#chuyển ảnh màu thành ảnh xám
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [53]:
#lọc Gaussian
smooth_img = cv2.GaussianBlur(gray_img,(3,3),0.5,cv2.BORDER_DEFAULT)

#lọc Bilateral
#smooth_img = cv2.bilateralFilter(gray_img,3,0.5,0.5,cv2.BORDER_DEFAULT)

In [54]:
#tính fx,fy bằng sobel
fx = cv2.Sobel(smooth_img, cv2.CV_64F, 1, 0, ksize=3)
fy = cv2.Sobel(smooth_img, cv2.CV_64F, 0, 1, ksize=3)

In [64]:
#tính fx,fy bằng roberts
# Roberts_Hx = np.array([[0,0,0],
#                        [0,1,0],
#                        [0,0,-1]])
# Roberts_Hy = np.array([[0,0,0],
#                        [0,0,1],
#                        [0,-1,0]])
# fx = ndimage.convolve(smooth_img, Roberts_Hx)
# fy = ndimage.convolve(smooth_img, Roberts_Hy)

In [57]:
#tính fx,fy theo prewitt
# Prewitt_Hx = np.array([[1,0,-1],
#                           [1,0,-1],
#                           [1,0,-1]])
# Prewitt_Hy = np.array([[1,1,1],
#                        [0,0,0],
#                        [-1,-1,-1]])

# fx = cv2.filter2D(smooth_img, -1, Prewitt_Hx)
# fy = cv2.filter2D(smooth_img, -1, Prewitt_Hy)

In [58]:
#độ lớn của Gradient
def Gradient_Magnitude(fx, fy): 
    mag = abs(fx) + abs(fy)
    return np.around(mag)
mag = Gradient_Magnitude(fx,fy)

In [59]:
#hướng của gradient
def Gradient_Direction(fx, fy):
    g_dir = np.arctan2(fy, fx)* 180 / np.pi
    return g_dir
dir = Gradient_Direction(fx,fy)

In [60]:
#ép hướng của gradient về 4 hướng
def Digitize_angle(Angle): 
    angle = np.abs(Angle)
    angle[angle <= 22.5] = 0
    angle[angle >= 157.5] = 0
    angle[(angle > 22.5) * (angle < 67.5)] = 1
    angle[(angle >= 67.5) * (angle <= 112.5)] = 2
    angle[(angle > 112.5) * (angle <= 157.5)] = 3
    return angle
dig = Digitize_angle(dir)

In [61]:
#loại pixel không phải cực đại 
def Non_Max_Supp(qn, magni):
    M = np.zeros(qn.shape)
    a = qn.shape[0]
    b = qn.shape[1]
    for i in range(a-1):
        for j in range(b-1):
            if qn[i,j] == 0:
                if  magni[i,j-1]< magni[i,j] or magni[i,j] > magni[i,j+1]:
                    M[i,j] = magni[i,j]
                else:
                    M[i,j] = 0
            if qn[i,j] == 1:
                if  magni[i-1,j+1]<= magni[i,j] or magni[i,j] >= magni[i+1,j-1]:
                    M[i,j] = magni[i,j]
                else:
                    M[i,j] = 0       
            if qn[i,j] == 2:
                if  magni[i-1,j]<= magni[i,j] or magni[i,j] >= magni[i+1,j]:
                    M[i,j] = magni[i,j]
                else:
                    M[i,j] = 0
            if qn[i,j] == 3:
                if  magni[i-1,j-1]<= magni[i,j] or magni[i,j] >= magni[i+1,j+1]:
                    M[i,j] = magni[i,j]
                else:
                    M[i,j] = 0
    return M
nms = Non_Max_Supp(dig,mag)

In [62]:
#đưa ảnh về 3 mức độ
def _double_thresholding(g_suppressed, low_threshold, high_threshold):
    g_thresholded = np.zeros(g_suppressed.shape)
    for i in range(0, g_suppressed.shape[0]):
        for j in range(0, g_suppressed.shape[1]):
            if g_suppressed[i,j] < low_threshold:
                g_thresholded[i,j] = 0
            elif g_suppressed[i,j] >= low_threshold and g_suppressed[i,j] < high_threshold: 
                g_thresholded[i,j] = 128
            else:
                g_thresholded[i,j] = 255
    return g_thresholded
dth = _double_thresholding(nms,100,200)

In [63]:
#bổ sung điểm ảnh
def _hysteresis(g_thresholded):
    g_strong = np.zeros(g_thresholded.shape)
    for i in range(0, g_thresholded.shape[0]):
        for j in range(0, g_thresholded.shape[1]):
            val = g_thresholded[i,j]
            if val == 128:
                if g_thresholded[i-1,j] == 255 or g_thresholded[i+1,j] == 255 or g_thresholded[i-1,j-1] == 255 or g_thresholded[i+1,j-1] == 255 or g_thresholded[i-1,j+1] == 255 or g_thresholded[i+1,j+1] == 255 or g_thresholded[i,j-1] == 255 or g_thresholded[i,j+1] == 255:
                    g_strong[i,j] = 255
            elif val == 255:
                g_strong[i,j] = 255
    return g_strong
result = _hysteresis(dth)